In [1]:
# 2025/7/17
# zhangzhong

# 我需要写一个可以按照一定的比例，混合多个数据集
# 带有shuffle
# 我们先用GPT2自带的tokenizer吧
# 之后再换成我们自己train出来的

from transformers import AutoTokenizer
from datasets import load_dataset, Dataset, IterableDataset

In [2]:
# 我们最终的目的是构造一个pytorch的Dataset
# 先不用想着DDP
# 先吧最简单的东西构造出来
# 那就先整一个数据集吧


tokenizer = AutoTokenizer.from_pretrained("gpt2")
print(tokenizer.eos_token)
eos_token_id: int = tokenizer.encode(tokenizer.eos_token)[0]
print(f"eos_token_id: {eos_token_id}")

<|endoftext|>
eos_token_id: 50256


In [3]:
# 首先就只拿一个数据集进行测试吧
# 我们还是用stream模式
# 只不过是本地的stream
dataset = load_dataset(
    path="wikimedia/wikipedia",
    name="20231101.en",
    # split="train",
    split="train[:1000]",
)
assert isinstance(dataset, Dataset)

Resolving data files:   0%|          | 0/41 [00:00<?, ?it/s]

In [4]:
print(dataset)

Dataset({
    features: ['id', 'url', 'title', 'text'],
    num_rows: 1000
})


In [5]:
tokenizer("text")

{'input_ids': [5239], 'attention_mask': [1]}

In [6]:
assert isinstance(dataset, Dataset)
iterable_dataset: IterableDataset = dataset.to_iterable_dataset()


# 然后就是对数据进行tokenization
# huggingface里面的有tokenization的例子
# 咱们参考一下
# 我觉得可以写一个函数，然后做batch处理
# 然后处理完成的tokenization
# 然后在load的时候，在做packing
# 分开做，感觉更好一些
def tokenize(examples):
    return tokenizer(examples["text"])


# TODO: 我现在怀疑这个tokenizer有长度限制，把超过长度的文本给截断了
tokenized_iterable_dataset = iterable_dataset.map(
    function=tokenize,
    batched=True,
    remove_columns=["id", "url", "title", "text"],
    # drop_last_batch=True,
    # https://huggingface.co/docs/datasets/process#multiprocessing
    # Dataset是支持多进程处理map的
    # 但是stream不支持
    # num_proc=4,  # 可以根据CPU核心数调整
)

In [7]:
# 统计一下token的总数量
token_counts: list[int] = []
token_count = 0
for batch in tokenized_iterable_dataset:
    token_count += len(batch["input_ids"])
    token_counts.append(len(batch["input_ids"]))
print(f"Total tokens: {token_count}")
print(f"Average tokens per example: {token_count / 1000}")
# 也不对，看看分布？
print(f"Token counts: {token_counts[:10]}...")  # 打印前10个
# 没有truncation
# ok，那就没问题了
# 为什么这里的features是unknown?
print(tokenized_iterable_dataset)

Token indices sequence length is longer than the specified maximum sequence length for this model (8668 > 1024). Running this sequence through the model will result in indexing errors


Total tokens: 5219402
Average tokens per example: 5219.402
Token counts: [8668, 5421, 2717, 16492, 11192, 18300, 13813, 2594, 548, 10647]...
IterableDataset({
    features: Unknown,
    num_shards: 1
})


In [ ]:
# TODO
# 不过我们可以手动做多进程支持
# 每个进程处理一个数据集，然后汇总到主进程，得到一个batch
# 再分发给不同的显卡
# 不过只有当我们的显卡占用率因为数据处理的太慢而占用率较低时，才应该使用

# 然后我们需要首先packing
# packing就是把一个batch里面的token 通过一个特殊的 eos token拼在一起
# 然后切分为1024的长度


def packing(examples):
    # 这里的examples是一个batch
    # 我们需要把每个example的tokens拼接起来
    # 然后切分为1024的长度
    # packed_texts = []
    # for text in examples["input_ids"]:
    #     packed_text = tokenizer.eos_token.join()
    #     packed_texts.append(packed_text)
    # list of int 要怎么做join ？
    packed_input_ids: list[int] = []
    for input_ids in examples["input_ids"]:
        packed_input_ids.extend(input_ids)
        packed_input_ids.append(eos_token_id)  # 添加eos token
    # 不对，没必要这么处理，加上了就加上了呗
    # packed_texts.append(examples["input_ids"][-1])  # 最后一个input_ids不需要添加eos token
    print(f"Packed input IDs length: {len(packed_input_ids)}")
    # 然后切分成1024的长度
    # 最后一行，如果不足1024，就不要了。
    cutted_packed_input_ids: list[list[int]] = []
    for i in range(0, len(packed_input_ids), 1024):
        cutted_packed_input_ids.append(packed_input_ids[i : i + 1024])
    # 去掉最后一个
    # cutted_packed_input_ids.pop(-1)  # 最后一行如果不足1024，就不要了。
    # 然后统计一下cutted_packed_input_ids的token数量
    total_tokens = sum(len(ids) for ids in cutted_packed_input_ids)
    print(f"Total tokens after packing: {total_tokens}")
    print(f"Number of packed sequences: {len(cutted_packed_input_ids)}")
    # 这里返回的就是一个batch

    return {"new_input_ids": cutted_packed_input_ids}

    # # 切分为1024的长度
    # packed_texts = [text[:1024] for text in packed_texts]

    # return {"input_ids": packed_texts}
    # return examples


print(tokenized_iterable_dataset.column_names)
packing_iterable_dataset = tokenized_iterable_dataset.map(
    function=packing,
    batched=True,
    # batch_size=8,  # 可以根据显存大小调整
    # 哈哈哈！huggingface的bug呀！怎么iterable dataset的features是unknown?
    # 这竟然不是bug！这是feature！
    # 因为stream模式，是lazy的
    # 除非我们去iter这个dataset，否则huggingface是拿不到schema 也就是列名的
    # 这种情况下，比较简单的处理方式，就是直接像我这样，显示的把列名给写出来
    remove_columns=["input_ids", "attention_mask"],
    # drop_last_batch=True,
)

None


In [9]:
# print(next(iter(iterable_dataset)))
# batch = next(iter(packing_iterable_dataset))
# print(len(batch['new_input_ids']))
lens = 0
for batch in packing_iterable_dataset:
    print(len(batch["new_input_ids"]))
    lens += 1
print(f"Total batches: {lens}")

Packed input IDs length: 5220402
Total tokens after packing: 5220402
Number of packed sequences: 5099
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
102

In [11]:
# 那这样的话，我们应该也可以用一个固定的batch_size来迭代这个数据集了
batched_dataset = packing_iterable_dataset.batch(
    batch_size=32, drop_last_batch=True
).with_format("torch")
lens = 0
for batch in batched_dataset:
    print(batch["new_input_ids"].shape)  # 打印每个batch的形状
    # 这里可以进行训练或者其他操作
    # break  # 只打印一个batch的形状
    lens += 32
print(f"Total examples in batched dataset: {lens}")

Packed input IDs length: 5220402
Total tokens after packing: 5220402
Number of packed sequences: 5099
torch.Size([32, 1024])
torch.Size([32, 1024])
torch.Size([32, 1024])
torch.Size([32, 1024])
torch.Size([32, 1024])
torch.Size([32, 1024])
torch.Size([32, 1024])
torch.Size([32, 1024])
torch.Size([32, 1024])
torch.Size([32, 1024])
torch.Size([32, 1024])
torch.Size([32, 1024])
torch.Size([32, 1024])
torch.Size([32, 1024])
torch.Size([32, 1024])
torch.Size([32, 1024])
torch.Size([32, 1024])
torch.Size([32, 1024])
torch.Size([32, 1024])
torch.Size([32, 1024])
torch.Size([32, 1024])
torch.Size([32, 1024])
torch.Size([32, 1024])
torch.Size([32, 1024])
torch.Size([32, 1024])
torch.Size([32, 1024])
torch.Size([32, 1024])
torch.Size([32, 1024])
torch.Size([32, 1024])
torch.Size([32, 1024])
torch.Size([32, 1024])
torch.Size([32, 1024])
torch.Size([32, 1024])
torch.Size([32, 1024])
torch.Size([32, 1024])
torch.Size([32, 1024])
torch.Size([32, 1024])
torch.Size([32, 1024])
torch.Size([32, 1024])
t

In [ ]:
# 然后可以和pytorch的dataloader集成
# 不行，接下来我需要和model结合调试，才能看到dataloader里面出来的是什么东西了
# ok！下午设计一下model structure！